# About
Here, I build a Q&A LLM chain to read the book "Time Machine" and answer questions about the book. 

The LLM model I ued is Meta's `llama2 7B` model (Ollma ID `78e26419b446`).

# Settings

### Packages

In [17]:
# Langchain related 
from langchain import PromptTemplate
from langchain.document_loaders import GutenbergLoader

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.llms import HuggingFaceHub


from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


import os
import sys

sys.path.append('../')

# put Huggingface API token in an py file (..config/token_access.py) and load it
from config import token_access

# set HUGGINGFACEHUB_API_TOKEN to my token_access
os.environ["HUGGINGFACEHUB_API_TOKEN"] = token_access.token_access


### Functions

In [10]:
class SuppressStdout:
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr


### Variables that require changes

In [ ]:
#----------------#
# data related
#----------------#
# a variable name used to define specific path
author="HGWells"

# book txt file link
book_link ="https://www.gutenberg.org/cache/epub/35/pg35.txt"

#----------------#
# model related
#----------------#
#  embeddings
embeddings_model_id ="huggingface_embeddings"

# llm model used for reading the book
llm_model_id ="llama2"

#----------------#
# your project root path 
#----------------#
main_Dir = "../book-reader"


### Directories

In [8]:
#----------------#
# data dir
#----------------#
data_Dir = os.path.join(main_Dir,"data")

# embeddings dir
embedding_string= f"{embeddings_model_id}".replace("/", "_").replace("-","_")

# hugging face llm pipeline model dir
embedding_Dir = os.path.join(data_Dir,f"{author}_{embedding_string}")

#----------------#
# model dir
#----------------#
model_Dir= os.path.join(main_Dir,"model")
cache_Dir = os.path.join(model_Dir,"cache")


#----------------#
# make dirs
#----------------#
for f in [data_Dir, embedding_Dir , model_Dir, cache_Dir]:
    os.makedirs(f, exist_ok=True)


# Read a book 
### "The Time Machine by H. G. Wells" from Gutenberg Project

In [9]:
# load the book 
book_loader = GutenbergLoader(book_link)  

document = book_loader.load()

## Creating a QA LLM Chain
This chain will be used to do QA on the document. We will need
 * A vector database that can perform document retrieval
 * An LLM to do the language interpretation
 * Specification on how to deal with this data 

### A vector database that can perform document retrieval
 * split data into chunks
 * use Hugging Face's embedding LLM to embed this data for our vector store

In [13]:
# chunk sizes
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(document)

# create embeddings
embeddings = HuggingFaceEmbeddings(
  cache_folder=embedding_Dir
)

# Make Index using chromadb and the embeddings LLM
with SuppressStdout():   
    chromadb_index = Chroma.from_documents(documents=texts, embedding=embeddings)


config_sentence_transformers.json: 100%|████████████████████████████████████████| 116/116 [00:00<00:00, 34.5kB/s]
README.md: 100%|████████████████████████████████████████████████████████████| 10.6k/10.6k [00:00<00:00, 3.22MB/s]
sentence_bert_config.json: 100%|██████████████████████████████████████████████| 53.0/53.0 [00:00<00:00, 12.3kB/s]
config.json: 100%|███████████████████████████████████████████████████████████████| 571/571 [00:00<00:00, 232kB/s]
pytorch_model.bin: 100%|██████████████████████████████████████████████████████| 438M/438M [01:36<00:00, 4.55MB/s]
tokenizer_config.json: 100%|████████████████████████████████████████████████████| 363/363 [00:00<00:00, 10.8kB/s]
vocab.txt: 100%|██████████████████████████████████████████████████████████████| 232k/232k [00:00<00:00, 1.18MB/s]
tokenizer.json: 100%|█████████████████████████████████████████████████████████| 466k/466k [00:00<00:00, 1.68MB/s]
1_Pooling/config.json: 100%|████████████████████████████████████████████████████| 190/19

### An LLM which does interpretation

In [47]:
# using Ollama 
hf_llm = Ollama(model=llm_model_id, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

### Template

In [23]:
# Prompt
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)


In [24]:
# Convert index to retriever (A wrapper around the functionality of our vector database so we can search for similar documents/chunks in the vectorstore and retrieve the results)
retriever = chromadb_index.as_retriever()

# qa retriever
qa_chain = RetrievalQA.from_chain_type(
    hf_llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)


# Q&A

### Did ok

In [32]:
question = "Who wrote this book?"
answer = qa_chain({"query": question})

The book was written by H.G. Wells.

In [26]:
question = "Who is the author of Time Machine?"
answer = qa_chain({"query": question})
display(answer)

The author of The Time Machine is H.G. Wells.

{'query': 'Who is the author of Time Machine?',
 'result': 'The author of The Time Machine is H.G. Wells.'}

In [28]:
question = "How do Eloi look like?"
answer = qa_chain({"query": question})

The text does not provide a clear description of how the Eloi look like. However, it is mentioned that they are "mere fatted cattle" and "disgusting" to the Morlocks, which suggests that they may be weak and vulnerable compared to their more robust and intelligent creators. The text also notes that the Eloi have retained some human form, but it is not clear what this means in terms of their physical appearance. Overall, the Eloi are portrayed as being inferior to the Morlocks in both physical strength and intelligence.

In [29]:
question = "How do Morlock look like?"
answer = qa_chain({"query": question})

Morlocks are described as "pallid bodies" with "battered" faces and "reddish" backs. They are also said to be "subterranean for innumerable generations," suggesting that they have evolved to live underground.

In [30]:
question = "In this novella, what is the fourth dimension?"
answer = qa_chain({"query": question})

The fourth dimension in the novella "The Time Machine" is a hypothetical direction perpendicular to the three dimensions of space that we are familiar with. According to the Time Traveller, consciousness moves intermittently in one direction along this fourth dimension from the beginning to the end of our lives. Some philosophical people have been asking why three dimensions particularly and have even tried to construct a Four-Dimensional geometry, but the Time Traveller believes that there is no difference between time and any of the three dimensions of space except for our consciousness moving along it.

In [31]:
question = "What temporarily blinds the Morlocks?"
answer = qa_chain({"query": question})

 The Morlocks are temporarily blinds by the glare of the fire.

In [33]:
question = "Who is Filby?"
answer = qa_chain({"query": question})

Filby is an argumentative person with red hair.

In [34]:
question = "What scientific principle does the Time Traveler's visit to the decaying world thirty millions in the future illustrate?"
answer = qa_chain({"query": question})

The scientific principle that the Time Traveler's visit to the decaying world thirty millions in the future illustrates is the concept of causality. The idea that events in the past can have a significant impact on the present, and that the actions of beings in the past can shape the course of events in the future. The Time Traveller's journey through time demonstrates how events that occurred in the past can have a lasting impact on the present, even after millions of years have passed.

In [35]:
question = "Where is the Time Machine held?"
answer = qa_chain({"query": question})


The Time Machine is held inside a pedestal.

In [36]:
question = "What do the Eloi eat?"
answer = qa_chain({"query": question})

The Eloi eat fruit.

In [38]:
question = "Who was the inventor of time machine in the novella?"
answer = qa_chain({"query": question})

The inventor of the time machine in the novella is the Time Traveller.

### Did not do well (Did not ask properly)

In [27]:
question = "Who invented the Time Machine?"
answer = qa_chain({"query": question})

 The Time Machine was invented by H.G. Wells.

In [49]:
question = "What is the name of the Time Traveller in the novella?"
answer = qa_chain({"query": question})

The name of the Time Traveller in the novella is "The Time Traveller."

# Save 

In [48]:
# save hugging face llm pipeline
hf_llm_string= f"{llm_model_id}".replace("/", "_").replace("-","_")

# hugging face llm pipeline model dir
hf_llm_Dir = os.path.join(model_Dir,f"{author}_{hf_llm_string}")

# create dir if not existing 
for f in [hf_llm_Dir]:
    os.makedirs(f, exist_ok=True)


# save huggingface pipeline 
hf_llm.save(os.path.join(hf_llm_Dir,f"{hf_llm_string}.json"))